# A deeper look into Python functions

## Python functions are objects

As usual, a Python function is a Python object

In [7]:
def f(x, y):
    pass

In [2]:
f.__class__

function

In [3]:
 issubclass(f.__class__, object)

True

This means for example that we can attach attributes to Python functions:

In [4]:
f.x = 2
f.__dict__

{'x': 2}

Use `dir` on any function to see the functions methods and attributes.

In [5]:
dir(f)

['__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__hash__',
 '__init__',
 '__module__',
 '__name__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'func_closure',
 'func_code',
 'func_defaults',
 'func_dict',
 'func_doc',
 'func_globals',
 'func_name',
 'x']

Calling the function invokes the `__call__` operator:

In [9]:
f.__call__(1, 2)

## Functions as arguments

Like all objects, functions can be arguments to functions

In [10]:
def add(x, y):
    return x+y

def sub(x, y):
    return x-y

def apply(func, x, y):
    return func(x, y)

In [11]:
apply(add, 1, 2)

3

In [12]:
apply(sub, 7, 5)

2

## Functions inside functions

Python allows nested function definitions:

In [17]:
def g(x, y):
    
    def cube(x):
        return x*x*x
    
    return y*cube(x)

g(4, 6)

dir(g)


['__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__hash__',
 '__init__',
 '__module__',
 '__name__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'func_closure',
 'func_code',
 'func_defaults',
 'func_dict',
 'func_doc',
 'func_globals',
 'func_name']

## Function returning functions

In [18]:
def h():
    
    def inner_h():
        print "Inside inner_h"
        
    return inner_h

foo = h()
foo

<function __main__.inner_h>

In [19]:
foo()

Inside inner_h


## More functions returning functions: *decorators*

A toy example

In [20]:
def foo():
    return 1

def outer(func):
    def inner():
        print "before calling func"
        return func()
    return inner

In [21]:
decorated = outer(foo)

The function `decorated` is a decorated version of function `foo`.
It is `foo` plus something more:

In [22]:
decorated()

before calling func


1

To simplify, we could just write
```python 
foo = outer(foo)
```
to replace foo with its decorated version each time it is called

## A (slightly) more useful decorator

Suppose we want to limit the range of values sent to a
mathematical formula:

In [23]:
from math import log
def f(x):
    return log(x) - 2

In [24]:
def checkrange(func):
    def inner(x):
        if x <= 0:
            print "out of range"
        else:
            return func(x)
    return inner

In [25]:
f(5)

-0.3905620875658997

In [26]:
f(-1)

ValueError: math domain error

In [27]:
f = checkrange(f)
f(5)

-0.3905620875658997

In [28]:
f(-1)

out of range


## The `@decorator` syntax

Python provides a short notation for decorating a function with
another function:

In [29]:
@checkrange
def g(x):
    return log(x) - 2

In [30]:
g(0)

out of range


This is essentially the same as writing `g = checkrange(g)`.

A decorator is simply a function taking another function as input
and returning another function. 

The syntax `@decorator` is a
short-cut for the more explicit `f = decorator(f)`.

## A (much) more useful decorator: memoization

Assume we have a slow function. Something like

In [31]:
from time import sleep

def slow(x, y):
    res = x*y; sleep(1)     # Simulate a long computation
    return res

In [32]:
%timeit slow(1, 2)

1 loops, best of 3: 1 s per loop


We call the function with the same input arguments, and hence perform the same (slow) calculations multiple times.

The idea of memoization (or buffering) is to buffer the input-output pairs for which the function was called.
If the function is called twice with same input arguments, we return the buffer value.

The implementation of a memoization with a `decorator` could look like:

In [33]:
cache = {}  # Stores all input-output pairs

def memoize(func):
    ''' Caches a function's return value each time it is called.
        If called later with the same arguments, the cached value is returned
        (not reevaluated). '''
    
    def inner(x, y):
        if (x, y) in cache:
            return cache[(x, y)]
        else:
            result = func(x, y)
            cache[(x, y)] = result
            return result
        
    return inner

Now we can apply the decorator to our slow function

In [34]:
@memoize
def slow(x, y):
    res = x*y; sleep(1)     # Simulate a long computation
    return res

In [37]:
print cache

{(1, 3): 3}


... and test it out

In [38]:
%timeit -r 1 -n 1 slow(1, 3)

1 loops, best of 1: 3.1 µs per loop


## Decorator summary 

* A function that takes a function as argument and returns a modified function
* `@decorator` syntax simply a short cut for the standard function call `f = decorator(f)`.